In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# experiment 1:
# first, creating a set of N randomly generated (m, 1) vectors:
import numpy as np
m = 5
N = 100
vectors = [np.random.rand(*(m, 1)) for _ in range(N)]
print (vectors)

[array([[0.47716303],
       [0.80583375],
       [0.49126072],
       [0.69828427],
       [0.39060723]]), array([[0.7437585 ],
       [0.42296043],
       [0.38942144],
       [0.26495093],
       [0.76262255]]), array([[0.14794172],
       [0.36198087],
       [0.30897352],
       [0.2928449 ],
       [0.97076944]]), array([[0.38000101],
       [0.59079602],
       [0.16126416],
       [0.11610269],
       [0.16789694]]), array([[0.54350361],
       [0.33422097],
       [0.60474248],
       [0.39090832],
       [0.98449557]]), array([[0.88330634],
       [0.82714442],
       [0.06917505],
       [0.79327687],
       [0.573112  ]]), array([[0.34807141],
       [0.17713913],
       [0.68056889],
       [0.81787864],
       [0.40656868]]), array([[0.74454203],
       [0.77736628],
       [0.5803259 ],
       [0.19379054],
       [0.93931984]]), array([[0.6704806 ],
       [0.4102489 ],
       [0.30625346],
       [0.04610873],
       [0.40348039]]), array([[0.6766924 ],
       [0.85132

In [3]:
# finding nearest neighbor of each vector using O(N^2) brute force approach:
nearest_neighbors = {} # the keys will be the indices of the vectors from 0 to N-1, and the corresponding values will be the indices of its nearest neighbor
for i in range(len(vectors)):
    nearest_dist = float('inf')
    nearest_neighbors[i] = None
    for j in range(len(vectors)):
        if j != i:
            dist = np.linalg.norm(vectors[i] - vectors[j])
            if dist < nearest_dist:
                nearest_dist = dist
                nearest_neighbors[i] = j
print (nearest_neighbors) 

{0: 41, 1: 32, 2: 85, 3: 10, 4: 24, 5: 64, 6: 23, 7: 24, 8: 29, 9: 50, 10: 3, 11: 41, 12: 81, 13: 93, 14: 65, 15: 85, 16: 84, 17: 98, 18: 55, 19: 9, 20: 86, 21: 28, 22: 57, 23: 6, 24: 7, 25: 32, 26: 90, 27: 70, 28: 50, 29: 8, 30: 85, 31: 70, 32: 1, 33: 56, 34: 63, 35: 26, 36: 89, 37: 70, 38: 94, 39: 90, 40: 58, 41: 0, 42: 81, 43: 74, 44: 46, 45: 81, 46: 44, 47: 63, 48: 77, 49: 53, 50: 9, 51: 54, 52: 62, 53: 60, 54: 51, 55: 18, 56: 72, 57: 22, 58: 23, 59: 43, 60: 53, 61: 9, 62: 52, 63: 87, 64: 5, 65: 14, 66: 73, 67: 56, 68: 86, 69: 80, 70: 27, 71: 88, 72: 80, 73: 66, 74: 43, 75: 90, 76: 97, 77: 99, 78: 71, 79: 74, 80: 69, 81: 12, 82: 6, 83: 37, 84: 92, 85: 2, 86: 20, 87: 63, 88: 71, 89: 36, 90: 88, 91: 74, 92: 53, 93: 99, 94: 78, 95: 87, 96: 50, 97: 31, 98: 17, 99: 77}


In [4]:
# finding nearest neighbor of each vector using NaiveLSH:
from memristor.engine.model import NaiveLSH
from memristor.crossbar.model import LineResistanceCrossbar
from memristor.devices import StaticMemristor
naive_lsh = NaiveLSH(
    hash_size=3, # adjustable hyperparameter
    crossbar_class=LineResistanceCrossbar,
    crossbar_params={'r_wl': 20, 'r_bl': 20, 'r_in':10, 'r_out':10, 'V_SOURCE_MODE':'|_|'},
    memristor_model_class=StaticMemristor,
    memristor_params={'frequency': 1e8, 'temperature': 273 + 40},
    m=m,
    r=1, # adjustable hyperparameter
)
reps = 5 # adjustable hyperparameter (repetitions of the hashing)
all_bins = []
for _ in range(reps):
    bins = {}
    for i in range(len(vectors)):
        hash = naive_lsh.inference(vectors[i])
        print (hash)
        if hash not in bins.keys():
            bins[hash] = [i]
        else:
            bins[hash].append(i)
    for bin in list(bins.values()):
        all_bins.append(bin)
#      {010:[1,5,7], 111:[5,6,7]}

# now at this point all_bins is a list like [[1,2], [1,3,5], ... ] where each element of all_bins is a bin containing indices of vectors that are likely
# to be close to each other. so now to find the nearest neighbor for each vector, we simply iterate through and check only those vectors that share a bin
# with it, so in this case for 1 we would check 2, 3, and 5, to find the nearest neighbor
nearest_neighbors_approx = {}
for i in range(len(vectors)):
    nearest_dist = float('inf')
    nearest_neighbors_approx[i] = None
    for bin in all_bins:
        if i in bin:
            for j in bin:
                if j != i:
                    dist = np.linalg.norm(vectors[i] - vectors[j])
                    if dist < nearest_dist:
                        nearest_dist = dist 
                        nearest_neighbors_approx[i] = j
print (nearest_neighbors_approx)

c:\Users\Ahsan\Desktop\pytorch-metal-oxide-memristor-crossbar\memristor\crossbar\model.py:31: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  self.fitted_w = torch.tensor([[self.memristors[i][j].g_linfit for j in range(ideal_w.shape[1])]


ValueError: operands could not be broadcast together with shapes (5,3) (3,6) 

In [5]:
print (nearest_neighbors)
print (nearest_neighbors_approx)

{0: 58, 1: 25, 2: 26, 3: 16, 4: 14, 5: 89, 6: 10, 7: 88, 8: 21, 9: 26, 10: 76, 11: 57, 12: 53, 13: 83, 14: 4, 15: 3, 16: 60, 17: 59, 18: 86, 19: 40, 20: 38, 21: 8, 22: 9, 23: 64, 24: 93, 25: 86, 26: 2, 27: 85, 28: 81, 29: 35, 30: 40, 31: 26, 32: 75, 33: 95, 34: 71, 35: 83, 36: 19, 37: 92, 38: 20, 39: 12, 40: 19, 41: 91, 42: 43, 43: 42, 44: 84, 45: 65, 46: 90, 47: 89, 48: 76, 49: 57, 50: 9, 51: 53, 52: 1, 53: 12, 54: 93, 55: 97, 56: 81, 57: 11, 58: 77, 59: 17, 60: 16, 61: 73, 62: 79, 63: 38, 64: 23, 65: 84, 66: 55, 67: 2, 68: 95, 69: 16, 70: 78, 71: 34, 72: 57, 73: 18, 74: 54, 75: 72, 76: 10, 77: 58, 78: 70, 79: 62, 80: 71, 81: 28, 82: 64, 83: 35, 84: 24, 85: 54, 86: 25, 87: 42, 88: 7, 89: 47, 90: 46, 91: 41, 92: 37, 93: 24, 94: 37, 95: 33, 96: 90, 97: 55, 98: 9, 99: 32}
{0: 58, 1: 86, 2: 26, 3: 16, 4: 14, 5: 34, 6: 10, 7: 88, 8: 21, 9: 26, 10: 76, 11: 49, 12: 53, 13: 83, 14: 4, 15: 3, 16: 60, 17: 59, 18: 86, 19: 40, 20: 38, 21: 8, 22: 9, 23: 64, 24: 93, 25: 73, 26: 2, 27: 46, 28: 81, 2

In [6]:
nearest_neighbors == nearest_neighbors_approx

False

In [7]:
print (len(all_bins))
print (all_bins)

15
[[0, 4, 8, 11, 14, 21, 25, 28, 41, 49, 50, 55, 58, 61, 66, 69, 73, 77, 81, 89, 91], [1, 2, 3, 5, 6, 7, 9, 10, 12, 13, 15, 16, 17, 18, 19, 20, 22, 23, 24, 26, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 47, 48, 51, 52, 53, 54, 56, 57, 59, 60, 62, 63, 64, 65, 67, 68, 70, 71, 72, 74, 75, 76, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 90, 92, 93, 94, 95, 96, 97, 98, 99], [27, 46], [0, 4, 8, 11, 14, 21, 25, 28, 41, 49, 50, 55, 58, 61, 66, 69, 73, 77, 81, 89, 91], [1, 2, 3, 5, 6, 7, 9, 10, 12, 13, 15, 16, 17, 18, 19, 20, 22, 23, 24, 26, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 47, 48, 51, 52, 53, 54, 56, 57, 59, 60, 62, 63, 64, 65, 67, 68, 70, 71, 72, 74, 75, 76, 78, 79, 80, 82, 83, 84, 85, 86, 87, 88, 90, 92, 93, 94, 95, 96, 97, 98, 99], [27, 46], [0, 4, 8, 11, 14, 21, 25, 28, 41, 49, 50, 55, 58, 61, 66, 69, 73, 77, 81, 89, 91], [1, 2, 3, 5, 6, 7, 9, 10, 12, 13, 15, 16, 17, 18, 19, 20, 22, 23, 24, 26, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40,

In [ ]:
# when experimenting with like a bigger dataset and stuff come up with  a metric to compare these 2 dicts
# also compare the runtime complexities, cuz its possible that its working so well because of sth wrong in the implementation whiich results in the runtime
# just being the same as the brute force method

In [9]:
'''
things to check: 
- sizes of the bins, should not be big because otherwise there would be no runtime improvement -> get metrics for reduction of search space/runtime or space complexity
- varying the parameters like N, m, hash size, etc. (try bigger/more data)
- using lineres_memristive_vmm not naive_memristive_vmm
- experiment with varying non-idealities
- experiment 2, create visualizations
- write section 4 of the paper    (rn highest priority is finalizing the experiment and section 4 of the paper)
- is the change made in StaticMemristor fine?
'''

'\nthings to check: \n- sizes of the bins, should not be big because otherwise there would be no runtime improvement -> get metrics for reduction of search space/runtime or space complexity\n- varying the parameters like N, m, hash size, etc. (try bigger/more data)\n- using lineres_memristive_vmm not naive_memristive_vmm\n'